## Outils pour la manipulation d'images et librairies.


In [3]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

In [27]:
def RGB_YCbCr(image):
    new_image = np.empty(image.shape, dtype = np.uint8)
    new_image[:,:,0] = 0.299*image[:,:,0] + 0.587*image[:,:,1] + 0.114*image[:,:,2]
    new_image[:,:,1] = -0.1687*image[:,:,0] - 0.3313*image[:,:,1] + 0.5*image[:,:,2] + 128
    new_image[:,:,2] = 0.5*image[:,:,0] - 0.4187*image[:,:,1] - 0.0813*image[:,:,2] + 128
    return new_image


def YCbCr_RGB(image):
    image = np.float64(image.copy())
    new_image = np.empty(image.shape, dtype = np.uint8)
    new_image[:,:,0] = np.clip(image[:,:,0] + 1.402*(image[:,:,2] - 128), 0, 255)
    new_image[:,:,1] = np.clip(image[:,:,0] - 0.34414*(image[:,:,1] - 128) - 0.71414*(image[:,:,2] - 128), 0, 255)
    new_image[:,:,2] = np.clip(image[:,:,0] + 1.772*(image[:,:,1] - 128), 0, 255)
    return new_image

def next_multiple(num, div):
    if num % div == 0:
        return num
    else:
        return ((num // div) + 1) * div

def padding(image):
    new_image = np.zeros((next_multiple(image.shape[0], 8),next_multiple(image.shape[1], 8),image.shape[2]), dtype = np.uint8)
    new_image[:image.shape[0],:image.shape[1]] = image
    return new_image

def crop(image):
    non_black_cols = np.where(image[:, -1, :] != 0)[0]
    non_black_rows = np.where(image[-1, :, :] != 0)[0]

    last_non_black_col = non_black_cols[-1] + 1 if len(non_black_cols) > 0 else image.shape[1]
    last_non_black_row = non_black_rows[-1] + 1 if len(non_black_rows) > 0 else image.shape[0]

    return image[:last_non_black_row, :last_non_black_col, :]


## Découpage en blocs et compression

## Écriture dans un fichier

## Décompression

## Tests 

In [28]:
test = load("test.png")
# Image.fromarray(test, mode = "RGB").show()
# Image.fromarray(RGB_YCbCr(test), mode = "YCbCr").show()
# Image.fromarray(YCbCr_RGB(RGB_YCbCr(test)), mode = "RGB").show()
Image.fromarray(crop(padding(test)), mode = "RGB").show()